Загружаем библиотеки

In [1]:
%matplotlib inline
#! pip install pymorphy2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
from tqdm import tqdm
from sklearn.metrics import *
import warnings
warnings.filterwarnings("ignore")

In [2]:
import gensim.downloader as api

## Загружаем данные

Данные записаны в формате `json`. Для его чтения воспользуемся библиотекой `json` и методом `open`.

In [3]:
#!unzip 'IMDB Dataset.csv.zip'

In [4]:
# Загружаем данные
try:
    data = pd.read_csv('IMDB Dataset.csv', encoding= "utf-8")
except:
    data = pd.read_csv('IMDB Dataset.csv', encoding= "ISO-8859-1")

In [5]:
data.head(10)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


Будем решать задачу классификации на 3 класса.

проверяем наличие NaN

In [6]:
data['review'].isnull().sum()

0

Проверяем дубли

In [7]:
data = data.drop_duplicates(subset=['review'], keep='first')
print(data)

                                                  review sentiment
0      One of the other reviewers has mentioned that ...  positive
1      A wonderful little production. <br /><br />The...  positive
2      I thought this was a wonderful way to spend ti...  positive
3      Basically there's a family where a little boy ...  negative
4      Petter Mattei's "Love in the Time of Money" is...  positive
...                                                  ...       ...
49995  I thought this movie did a down right good job...  positive
49996  Bad plot, bad dialogue, bad acting, idiotic di...  negative
49997  I am a Catholic taught in parochial elementary...  negative
49998  I'm going to have to disagree with the previou...  negative
49999  No one expects the Star Trek movies to be high...  negative

[49582 rows x 2 columns]


In [8]:
#print(set([x["rating"] for x in data]))
data.dtypes

review       object
sentiment    object
dtype: object

In [9]:
data = data.astype({"sentiment": "string"})
data = data.astype({"review": "string"})

In [10]:
data.dtypes

review       string
sentiment    string
dtype: object

In [11]:
data.head(5)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [12]:
col_bytes_len = data['review'].max()
len(col_bytes_len)

679

In [13]:
rating_counts = data.groupby('sentiment')
print("Количество отзывов для каждого рейтинга:")
print(rating_counts.size())

Количество отзывов для каждого рейтинга:
sentiment
negative    24698
positive    24884
dtype: int64


# Предобработка данных

In [14]:
import nltk   # Natural Language Toolkit

In [15]:
# загружаем список стоп-слов для русского
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('english')

# примеры стоп-слов
print(len(stop_words))
print(stop_words[:10])

179
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/kbudakovskiy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Инициализируем `WordPunctTokenizer`, с помощью которого затем разобьем текст на слова.

In [16]:
word_tokenizer = nltk.WordPunctTokenizer()

Запишем предобработку текста в виде функции.

In [17]:
import re
regex = re.compile(r'[A-z]+')

def words_only(text, regex=regex):
    try:
        return " ".join(regex.findall(text)).lower()
    except:
        return ""

# расширим список стоп-слов, словами, которые являеются стоп-словами в данной задаче
add_stop_words = ['br']
all_stop_words = stop_words + add_stop_words


def process_data(data):
    texts = []
    targets = []

    # поочередно проходим по всем новостям в списке
    for item in tqdm(data):

        text_lower = words_only(item) # оставим только слова
        tokens     = word_tokenizer.tokenize(text_lower) #разбиваем текст на слова

        # удаляем пунктуацию и стоп-слова
        tokens = [word for word in tokens if (word not in all_stop_words and not word.isnumeric())]

        texts.append(tokens) # добавляем в предобработанный список

    return texts

In [18]:
# запускаем нашу предобработку
y = data["sentiment"].tolist()
texts = process_data(data["review"])

100%|██████████| 49582/49582 [00:10<00:00, 4897.62it/s]


In [19]:
# example
i = 36
print("Label: ", y[i])
print("Tokens: ", texts[i][:5])

Label:  negative
Tokens:  ['plot', 'death', 'little', 'children', 'hopper']


In [20]:
# загружаем библиотеку для лемматизации
import pymorphy2 # Морфологический анализатор

# инициализируем лемматизатор :)
morph = pymorphy2.MorphAnalyzer()

Посмотрим на примерах, как работает лемматизация.

In [21]:
i = 1
for aword in texts[i][:10]:
    aword_norm = morph.parse(aword)[0].normal_form
    print("Исходное слово: %s\tЛемматизированное: %s" % (aword, aword_norm))

Исходное слово: wonderful	Лемматизированное: wonderful
Исходное слово: little	Лемматизированное: little
Исходное слово: production	Лемматизированное: production
Исходное слово: filming	Лемматизированное: filming
Исходное слово: technique	Лемматизированное: technique
Исходное слово: unassuming	Лемматизированное: unassuming
Исходное слово: old	Лемматизированное: old
Исходное слово: time	Лемматизированное: time
Исходное слово: bbc	Лемматизированное: bbc
Исходное слово: fashion	Лемматизированное: fashion


In [22]:
# применяем лемматизацию ко всем текстам
for i in tqdm(range(len(texts))):           # tqdm_notebook создает шкалу прогресса :)
    text_lemmatized = [morph.parse(x)[0].normal_form for x in texts[i]] # применяем лемматизацию для каждого слова в тексте
    texts[i] = ' '.join(text_lemmatized)                # объединяем все слова в одну строку через пробел

 50%|████▉     | 24586/49582 [00:26<00:26, 945.53it/s] 

Посмотрим на пример.

In [ ]:
# посмотрим на пример
i = 1
print("Label: ",   y[i])
print("Text: \n",  texts[i])

# Моделирование & Векторные представления

## Разбиваем на train&test

Лейблы у нас также закодированы словами. Для корректной работы алгорима конвертируем их в числа (`'negative', 'neutral', 'positive'`):

    negative = -1
    neutral  = 0
    positive = 1

In [ ]:
# Функция для кодирования лейблов
def label2num(y):
    if y == 'positive':
        return 1
    if y == 'negative':
        return -1

encoded_y = [label2num(yy) for yy in y]

In [ ]:
print(encoded_y)

**Эффективность алгоритма некорректно оценивать на обучающих данных!** Это все равно что на контрольной ученику давать задачи, разобранные в классе.

Поэтому мы отложим часть данных для тестирования и оценки качества алгоритма. Для этого воспользуемся функцией `train_test_split`.

In [ ]:
#train test_split
from sklearn.model_selection import train_test_split
train_texts, test_texts, train_y, test_y = train_test_split(texts, encoded_y, test_size=0.2, random_state=42, stratify = y)

In [ ]:
#Инициализируем векторайзер
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features = 100)
vectorizer.fit(train_texts)

# Топ-10 слов
vectorizer.get_feature_names_out()[:10]

In [ ]:
# Обучаем vectorizer на train-данных и сразу преобразем их в вектора с помощью метода fit_transform
train_X = vectorizer.transform(train_texts)
train_X.todense()[:2]

In [ ]:
test_X  = vectorizer.transform(test_texts)

In [ ]:
#import алгоритма из библиотеки
from sklearn.ensemble import RandomForestClassifier

# инициализируем модель
clf = RandomForestClassifier(n_estimators = 500, max_depth = 10)

# обучаем ее на тренировочных данных
clf = clf.fit(train_X, train_y)

# делаем предсказание для тестовых данных
pred = clf.predict(test_X)

### Оценка качества

Качество классификатора будем оценивать по метрикам accuracy и f1.



In [ ]:
print('Accuracy: ', accuracy_score(test_y, pred))
print('F1: ', f1_score(test_y, pred, average = 'macro'))

# Посмотрим на несколько примеров

In [ ]:
for i in range(10):
    print('Истинный лейбл:',decoded_test_y[i])
    print('Предсказанный лейбл:',decoded_pred[i])
    print('Текст новости: ', train_texts[i][:500]+'...')
    print('\n')

In [ ]:
#вычисляем tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer
# Fit TF-IDF on train texts
vectorizer = TfidfVectorizer(max_features = 5000, norm = None) # возмем топ 200 слов
vectorizer.fit(train_texts)

# Топ-10 слов
vectorizer.get_feature_names_out()[:10]

In [ ]:
# Обучаем TF-IDF на train, а затем применяем к train и test
train_X = vectorizer.fit_transform(train_texts)
test_X  = vectorizer.transform(test_texts)

In [ ]:
# Пример
train_X.todense()[:2] # посмотрим на первые 2 строки

## Обучаем классификатор

In [ ]:
#import алгоритма из библиотеки
from sklearn.ensemble import RandomForestClassifier

# инициализируем модель
clf = RandomForestClassifier(n_estimators = 500, max_depth = 10)

# обучаем ее на тренировочных данных
clf = clf.fit(train_X, train_y)

# делаем предсказание для тестовых данных
pred = clf.predict(test_X)

In [ ]:
print('Accuracy: ', accuracy_score(test_y, pred))
print('F1: ', f1_score(test_y, pred, average = 'macro'))

## Посмотрим на несколько примеров

In [ ]:
for i in range(10):
    print('Истинный лейбл:',decoded_test_y[i])
    print('Предсказанный лейбл:',decoded_pred[i])
    print('Текст новости: ', train_texts[i][:500]+'...')
    print('\n')

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
        'n_estimators': range(470, 490, 1),
        'max_depth': range(5, 15, 5)
}

rfc = RandomForestClassifier()

grid_search = GridSearchCV(rfc, param_grid, cv=5, scoring='f1', verbose=1)
grid_search.fit(train_X, train_y)

print("Best CV score: {:.3f}, best CV n_estimators: {}".format(
    grid_search.best_score_, grid_search.best_estimator_.n_estimators)
) 


test_predictions = grid_search.best_estimator_.predict(test_X)
print("Resulting test score: {:.3f}".format(f1_score(test_predictions, test_y)))

In [ ]:
wv = api.load('word2vec-google-news-300')

In [ ]:
tfidf_weights = train_X.toarray()
tfidf_weights

Из чата GPT

In [ ]:
# Получение tf-idf весов
tfidf_weights = train_X.toarray()

# Векторизация текста с весами tf-idf и word2vec
def vectorize_with_tfidf_and_word2vec(text, tfidf_weights, word2vec_model):
    word_vectors = []
    words = text.split()
    for word in words:
        if word in word2vec_model:
            # Получение вектора word2vec для слова
            w2v_vector = word2vec_model[word]
            # Получение tf-idf веса для слова
            tfidf_weight = tfidf_weights[words.index(word)]
            # Умножение вектора word2vec на tf-idf вес
            weighted_w2v_vector = w2v_vector * tfidf_weight
            word_vectors.append(weighted_w2v_vector)
    
    if word_vectors:
        # Усреднение векторов слов для получения вектора предложения
        sentence_vector = np.mean(word_vectors, axis=0)
        return sentence_vector
    else:
        # Если нет слов из word2vec модели в тексте, вернуть нулевой вектор
        return np.zeros_like(word2vec_model['word'])

# Применение функции к каждому тексту в train_text
train_vectors = [vectorize_with_tfidf_and_word2vec(text, tfidf_weights[i], wv) for i, text in enumerate(train_texts)]

# Применение функции к каждому тексту в test_texts
test_vectors = [vectorize_with_tfidf_and_word2vec(text, tfidf_weights[i], wv) for i, text in enumerate(test_texts)]

In [ ]:
#import алгоритма из библиотеки
from sklearn.ensemble import RandomForestClassifier

# инициализируем модель
clf = RandomForestClassifier(n_estimators = 500, max_depth = 10)

# обучаем ее на тренировочных данных
clf = clf.fit(train_vectors, train_y)

# делаем предсказание для тестовых данных
pred = clf.predict(test_vectors)

In [ ]:
print('Accuracy: ', accuracy_score(test_y, pred))
print('F1: ', f1_score(test_y, pred, average = 'macro'))